# Simulation 10: User Mobility Reasonableness
TJ Kim
1/2/21

### Summary:
Goal is to show that the accuracy of the mobility model deteriorates with a longer look ahead window. This allows us to show that the mobility model and the cost saves are "realistic" in that our predictions are somewhat flawed, and our results are robust.

### Settings:

- 20 users
- 10/20 servers

In [1]:
from IPython.core.debugger import set_trace

# Import Generic Classes
import numpy as np
import copy
import pickle
import random
import itertools

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/classes")
sys.path.append(os.path.pardir+"/solvers")
sys.path.append(os.path.pardir+"/one_mobility")
sys.path.append(os.path.curdir + "/sim_store/sim10")
sys.path.append(os.path.pardir+"/crawdad_mobility")



from Server import *
from User import *
from Link import *
from Job import *
from Migration_Plans import *
from Cost_Recordings import *

from sim10_setting import *
from load_crawdad_usr import *


In [2]:
# Load Crawdad data
# Load All 12 users and trim their info
db_names = []
for i in range(1,13):
    db_names += ['LifeMap_GS'+str(i)+'.db']

db = []
for name in db_names:
    name = '/home/tkim/CMU/MEC_research/crawdad_mobility/' + name
    df,trace = load_usr(name)
    db += [trace]

Function for recording probability table.

In [3]:
def record_prob(num_server,num_user):
    """
    Record the accuracy of the markovian maximum prediction per timestep model
    For a single trial.
    """
    
    users, servers, links, jobs, sim_param = simulation_setting(db, num_server,num_user)
    users[0].user_voronoi_true.shape
    timesteps = users[0].user_voronoi_true.shape[0]
    accuracies = np.zeros(timesteps)
    
    for t in range(timesteps):
        for u in range(len(users)):
            pred_loc = np.argmax(users[u].server_prob[:,t])
            if users[u].user_voronoi_true[t] == pred_loc:
                accuracies[t] += 1
        
        accuracies[t] = np.divide(accuracies[t],len(users))
    
    return accuracies

In [4]:
# Testing performance
num_server = 10
num_user = 20
record_prob(num_server,num_user)

array([1.  , 0.85, 0.75, 0.5 , 0.6 , 0.5 , 0.35, 0.3 , 0.2 , 0.2 , 0.2 ,
       0.2 , 0.2 , 0.15, 0.15, 0.15, 0.05, 0.05, 0.15, 0.15])

### Test 1:

Record accuracy table for 20 users, 5 servers across 20 trials.

In [5]:
num_server = 5
num_user = 20
num_trial = 20
timesteps = 20

acc_temp = np.zeros(timesteps)

for i in range(num_trial):
    acc_temp += np.divide(record_prob(num_server,num_user),num_trial)
    
print(acc_temp)

[1.     0.8725 0.7775 0.7    0.6475 0.6275 0.5775 0.5525 0.5325 0.5
 0.49   0.4675 0.44   0.415  0.385  0.3625 0.32   0.295  0.2575 0.24  ]


### Test 2: 

20 users, 10 servers, 20 trials

In [6]:
num_server = 10
num_user = 20
num_trial = 20
timesteps = 20

acc_temp = np.zeros(timesteps)

for i in range(num_trial):
    acc_temp += np.divide(record_prob(num_server,num_user),num_trial)
    
print(acc_temp)

[1.     0.8025 0.69   0.6075 0.5225 0.49   0.475  0.45   0.4325 0.4
 0.3875 0.3375 0.315  0.3    0.28   0.26   0.2425 0.16   0.135  0.1175]


### Test 2:

Record accuracy table for 20 users, 20 servers, across 20 trials.

The accuracy should get lower faster due to higher server density.

In [7]:
num_server = 20
num_user = 20
num_trial = 20
timesteps = 20

acc_temp = np.zeros(timesteps)

for i in range(num_trial):
    acc_temp += np.divide(record_prob(num_server,num_user),num_trial)
    
print(acc_temp)

[1.     0.7125 0.58   0.5025 0.44   0.4    0.335  0.3425 0.295  0.2725
 0.25   0.2475 0.235  0.22   0.1925 0.16   0.145  0.1475 0.125  0.1425]
